In [7]:
import pandas as pd
from scipy import stats
import joblib
import matplotlib.pyplot as plt
import os

path = os.getcwd()

print(path)

C:\Users\Yusuke.Aoki\Documents\LocalEY\事例集\cibook


In [8]:
mail_df = pd.read_csv('Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [9]:
mail_df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [10]:
mail_df.describe()

,recency,history,mens,womens,newbie,visit,conversion,spend
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000
mean,5.763734,242.085656,0.551031,0.549719,0.502250,0.146781,0.009031,1.050908
std,3.507592,256.158608,0.497393,0.497526,0.499999,0.353890,0.094604,15.036448
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,64.660000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,158.110000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,9.000000,325.657500,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000,1.000000,499.000000


## <font color="royalblue">データ準備 </font> 

In [11]:
mail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   recency          64000 non-null  int64  
 1   history_segment  64000 non-null  object 
 2   history          64000 non-null  float64
 3   mens             64000 non-null  int64  
 4   womens           64000 non-null  int64  
 5   zip_code         64000 non-null  object 
 6   newbie           64000 non-null  int64  
 7   channel          64000 non-null  object 
 8   segment          64000 non-null  object 
 9   visit            64000 non-null  int64  
 10  conversion       64000 non-null  int64  
 11  spend            64000 non-null  float64
dtypes: float64(2), int64(6), object(4)
memory usage: 5.9+ MB


In [13]:
mail_df.segment.unique()

array(['Womens E-Mail', 'No E-Mail', 'Mens E-Mail'], dtype=object)

In [19]:
mail_df.channel.unique()

array(['Phone', 'Web', 'Multichannel'], dtype=object)

In [14]:
male_df=mail_df[mail_df.segment != 'Womens E-Mail'].copy()  #女性向けメールが配信されたデータを削除

In [15]:
male_df['treatment']=male_df.segment.apply(lambda x: 1 if x=='Mens E-Mail' else 0) #介入を表すtreatment変数を追加

## <font color="royalblue">集計 </font> 

In [17]:
#group_by, summariseを使って集計
male_df.groupby('treatment').agg(#データグループ化
    conversion_rate=('conversion','mean'), #グループごとのconversionの平均
    spend_mean=('spend','mean'), #グループごとのspendの平均
    count=('treatment','count') #グループごとのデータ数
    )

,conversion_rate,spend_mean,count
treatment,,,
0,0.005726,0.652789,21306
1,0.012531,1.422617,21307


### <font color="royalblue">t検定 </font>  

In [18]:
# t検定
### (a）男性向けメールが配信されたグループの購買データを得る
mens_mail=male_df[male_df.treatment==1].spend.values

### (b) メールが配信されなったグループの購買データを得る
no_mail=male_df[male_df.treatment==0].spend.values

### (a)(b)の平均の差に対して有意差検定を実行する
stats.ttest_ind(mens_mail,no_mail)

Ttest_indResult(statistic=5.300090294465472, pvalue=1.163200872605869e-07)

In [ ]:
#有意差はある模様


### <font color="royalblue">セレクションバイアスのあるデータで平均を比較 </font>  

In [20]:
#セレクションバイアスのあるデータの作成
sample_rules=(male_df.history>300)|(male_df.recency<6)|(male_df.channel=='Multichannel')

In [21]:
biased_df = pd.concat([
    male_df[(sample_rules) & (male_df.treatment == 0)].sample(frac=0.5, random_state=1),
    male_df[(sample_rules) & (male_df.treatment == 1)],
    male_df[(~sample_rules) & (male_df.treatment == 0)],
    male_df[(~sample_rules) & (male_df.treatment == 1)].sample(frac=0.5, random_state=1)
], axis=0, ignore_index=True)

In [22]:

biased_df.groupby('treatment').agg( # データのグループ化
    conversion_rate=('conversion', 'mean'), # グループごとのconversionの平均
    spend_mean=('spend', 'mean'), # グループごとのspendの平均
    count=('treatment', 'count') # グループごとのデータ数
)
# 乱数でbiased_dfのデータを作成しているので数値は書籍とは異なる。（結論は変わらない）

,conversion_rate,spend_mean,count
treatment,,,
0,0.004540,0.557954,14757
1,0.013572,1.541704,17168
